## StardewValley Wiki Scrapper



### Setup

In [2]:
import stars
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import regex


### Infox to DF

In [3]:
soup = BeautifulSoup(re.get('https://stardewvalleywiki.com/Fish_Taco').text)

In [4]:
table = soup.find('table', {'id':"infoboxtable"})
table_lines = table.find_all('tr')

In [5]:
table_lines = table.find_all('tr')

In [6]:
data = {'lines': []}

detail_flag = True
image_flag = True

for line in table_lines:
    
    # Finding the category the line is in
    if line.find('td', {"style":"text-align:center; padding-left: 0; padding-right: 0;"}) != None:
        cat_holder = line.find('td', {"style":"text-align:center; padding-left: 0; padding-right: 0;"}).text.replace('\n', '')

    # Finding the name of the item that the page is about
    elif line.find('td', {'id':"infoboxheader"}) != None:
        data['name'] = line.find('td', {'id':"infoboxheader"}).text.replace('\n', '')
    
    # Finding the image of the item that the page is about
    elif (line.find('img') != None) and (image_flag):
        data['image'] = 'https://stardewvalleywiki.com/' + line.find('img')['src']
        image_flag = False # This is to make sure that the image is only found once

    # Finding the description of the item that the page is about
    elif (line.find('td', {'id':"infoboxdetail"}) != None) and detail_flag:
        data['description'] = line.find('td', {'id':"infoboxdetail"}).text.replace('\n', '')
        detail_flag = False # This is to make sure that the description is only found once

    # Finding the contents of the infobox lines  
    elif line.find('td', {'id': 'infoboxsection'}) != None:
        section = line.find('td', {'id': 'infoboxsection'}).text.replace('\n', '')
        detail = line.find('td', {'id': 'infoboxdetail'}).text.replace('\xa0', '').replace('\n', '').replace('\t', '')

        # This is specific for the fields with images and values
        if section in ['Energy / Health', 'Qi Seasoning']:
            detail = stars.items.extract_text_before_image(line)
        
        # This is specific for the ingredients field
        if section == 'Ingredients':
            detail = stars.items.ectract_ingredients(line)

        data['lines'].append({section: detail})
        

In [7]:
data

{'lines': [{'Source': 'Cooking'},
  {'Buff(s)': ' Fishing(+2)'},
  {'Buff Duration': ' 7m'},
  {'Energy / Health': [['Energy', '165'], ['Health', '74']]},
  {'Sell Price': '500g'},
  {'Qi Seasoning': [['Energy', '297'],
    ['Health', '133'],
    ['Fish Taco', '750g']]},
  {'Recipe Source(s)': ' Linus(Mail - 7+ )'},
  {'Ingredients': [['Tuna', '1'],
    ['Tortilla', '1'],
    ['Red Cabbage', '1'],
    ['Mayonnaise', '1']]}],
 'name': 'Fish Taco',
 'image': 'https://stardewvalleywiki.com//mediawiki/images/d/d5/Fish_Taco.png',
 'description': 'It smells delicious.'}